In [1]:
import torch

In [2]:
def activation(x):
    return 1 / (1 + torch.exp(-x))

In [3]:
torch.manual_seed(7)

In [4]:
features = torch.randn((1, 5))
weights = torch.randn_like(features)
bias = torch.randn((1, 1))

In [5]:
output = activation((features * weights).sum() + bias)

In [6]:
torch.manual_seed(7)

features = torch.randn((1, 3))
n_input = features.shape[1]
n_hidden = 2
n_output = 1

w1 = torch.randn(n_input, n_hidden)
w2 = torch.randn(n_hidden, n_output)

b1 = torch.randn(1, n_hidden)
b2 = torch.randn(1, n_output)

out1 = activation(torch.mm(features, w1) + b1)
out2 = activation(torch.mm(out1, w2) + b2)


In [7]:
### Solution

h = activation(torch.mm(features, w1) + b1)
output = activation(torch.mm(h, w2) + b2)
print(output)

tensor([[0.3171]])


In [8]:
output.numpy()

array([[0.31708315]], dtype=float32)

In [9]:
from torchvision import datasets, transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)),])

trainset = datasets.MNIST('/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle = True)


0it [00:00, ?it/s]


Extracting /MNIST_data/MNIST\raw\train-images-idx3-ubyte.gz


FileNotFoundError: [Errno 2] No such file or directory: '/MNIST_data/MNIST\\raw\\train-images-idx3-ubyte.gz'

In [ ]:
dataiter = iter(trainloader)

images, labels = dataiter.next()
print(images.shape, labels.shape)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(images[4].numpy().squeeze(), cmap='Greys_r');
plt.show()

In [ ]:
inputs = images.view(images.shape[0], -1)
# print(inputs.shape)

w1 = torch.randn(784, 256)
w2 = torch.randn(256, 10)

b1 = torch.randn(256)
b2 = torch.randn(10)

h = activation(torch.mm(inputs, w1) + b1)
out = torch.mm(h, w2) + b2


In [ ]:
def softmax(x):
    return torch.exp(x) / torch.sum(torch.exp(x), dim = 1).view(-1, 1)

probabilities = softmax(out)
print(probabilities.shape)

In [ ]:
from torch import nn
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Inputs to hidden layer linear transformation
        self.hidden = nn.Linear(784, 256)
        # Output layer, 10 units - one for each digit
        self.output = nn.Linear(256, 10)
        
        # Define sigmoid activation and softmax output 
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        # Pass the input tensor through each of our operations
        x = self.hidden(x)
        x = self.sigmoid(x)
        x = self.output(x)
        x = self.softmax(x)
        
        return x

In [ ]:
# Create the network and look at it's text representation
model = Network()
model

In [ ]:
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        # Inputs to hidden layer linear transformation
        self.hidden = nn.Linear(784, 256)
        # Output layer, 10 units - one for each digit
        self.output = nn.Linear(256, 10)
        
    def forward(self, x):
        # Hidden layer with sigmoid activation
        x = F.sigmoid(self.hidden(x))
        # Output layer with softmax activation
        x = F.softmax(self.output(x), dim=1)
        
        return x

In [ ]:
model = Network()
model

In [ ]:
## Solution

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        # Defining the layers, 128, 64, 10 units each
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        # Output layer, 10 units - one for each digit
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        ''' Forward pass through the network, returns the output logits '''
        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.softmax(x, dim=1)
        
        return x

model = Network()
model

In [ ]:
print(model.fc1.weight)


In [ ]:
#fill all biases to 0
model.fc1.bias.data.fill_(0)


In [ ]:
#normalize weights with random variables of std 0.01
model.fc1.weight.data.normal_(0.01)

In [ ]:


# Grab some data 
dataiter = iter(trainloader)
images, labels = dataiter.next()

# Resize images into a 1D vector, new shape is (batch size, color channels, image pixels) 
images.resize_(64, 1, 784)
# or images.resize_(images.shape[0], 1, 784) to automatically get batch size

# Forward pass through the network
img_idx = 0
ps = model.forward(images[img_idx,:])

img = images[img_idx]



In [ ]:
import helper

helper.view_classify(img.view(1, 28, 28), ps)

In [ ]:
# Hyperparameters for our network
input_size = 784
hidden_sizes = [128, 64]
output_size = 10

# Build a feed-forward network
model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.Softmax(dim=1))
print(model)

# Forward pass through the network and display output
images, labels = next(iter(trainloader))
images.resize_(images.shape[0], 1, 784)
ps = model.forward(images[0,:])

In [ ]:
print(model[0])
model[0].weight

In [ ]:
from collections import OrderedDict
model = nn.Sequential(OrderedDict([
                      ('fc1', nn.Linear(input_size, hidden_sizes[0])),
                      ('relu1', nn.ReLU()),
                      ('fc2', nn.Linear(hidden_sizes[0], hidden_sizes[1])),
                      ('relu2', nn.ReLU()),
                      ('output', nn.Linear(hidden_sizes[1], output_size)),
                      ('softmax', nn.Softmax(dim=1))]))
model

In [ ]:
print(model[0])
print(model.fc1)

In [ ]:
import torch
from torchvision import datasets, transforms
# import helper

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [ ]:
from torch import nn, optim
import torch.nn.functional as F

class Classifier(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim = 1)
        
        return x
        

In [ ]:
model = Classifier()
images, labels = next(iter(trainloader))
criterion = nn.NLLLoss()

optimizer = optim.Adam(model.parameters(), lr = 0.003)

In [ ]:
epochs = 5

for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")

In [ ]:
images, labels = next(iter(testloader))
res = torch.exp(model(images))
print(res.shape)

In [ ]:
top_p, top_class = res.topk(1, dim = 1)
print(top_class[:10, ])

In [ ]:
equals = top_class == labels.view(*top_class.shape) 

In [ ]:
print(equals.type)

In [ ]:
accuracy = torch.mean(equals.type(torch.FloatTensor))
print(f'Accuracy: {accuracy.item()*100}%')
print(f'{accuracy.type}')